# Evolver Loop 5 Analysis

## Situation Summary
- 5 experiments, all at 70.676102 (no improvement)
- Target: 68.919154 (gap: 1.757 points = 2.55%)
- 93 submissions remaining

## What's Been Tried
1. exp_000: Baseline from pre-optimized CSV
2. exp_001: Full ensemble from 30 CSVs - same score
3. exp_002: Deletion cascade - no improvement
4. exp_003: Random initialization - much worse
5. exp_004: bbox3 optimization + lattice init - no improvement

## Key Insight from Evaluator
The evaluator correctly identified that:
1. bbox3 was run with too few iterations (10k vs 50k needed)
2. Lattice configs were NOT optimized with SA (just raw placement)
3. The seshurajup kernel has a sophisticated parallel SA optimizer that should be tried

In [1]:
import pandas as pd
import numpy as np
import os

# Load baseline
baseline_path = '/home/code/experiments/001_baseline/santa-2025.csv'
df = pd.read_csv(baseline_path, dtype=str)

# Tree geometry
TX = [0, 0.125, 0.0625, 0.2, 0.1, 0.35, 0.075, 0.075, -0.075, -0.075, -0.35, -0.1, -0.2, -0.0625, -0.125]
TY = [0.8, 0.5, 0.5, 0.25, 0.25, 0, 0, -0.2, -0.2, 0, 0, 0.25, 0.25, 0.5, 0.5]

import math

def get_vertices(x, y, deg):
    rad = deg * math.pi / 180
    c, s = math.cos(rad), math.sin(rad)
    vx = [x + TX[i] * c - TY[i] * s for i in range(15)]
    vy = [y + TX[i] * s + TY[i] * c for i in range(15)]
    return vx, vy

def get_side(trees):
    all_x, all_y = [], []
    for x, y, deg in trees:
        vx, vy = get_vertices(x, y, deg)
        all_x.extend(vx)
        all_y.extend(vy)
    return max(max(all_x) - min(all_x), max(all_y) - min(all_y))

def load_configs(df):
    configs = {}
    for n in range(1, 201):
        prefix = f'{n:03d}_'
        rows = df[df['id'].str.startswith(prefix)]
        trees = []
        for _, row in rows.iterrows():
            x = float(str(row['x']).replace('s', ''))
            y = float(str(row['y']).replace('s', ''))
            deg = float(str(row['deg']).replace('s', ''))
            trees.append((x, y, deg))
        configs[n] = trees
    return configs

configs = load_configs(df)
scores = {n: get_side(configs[n])**2 / n for n in range(1, 201)}
total = sum(scores.values())
print(f'Baseline total: {total:.6f}')
print(f'Target: 68.919154')
print(f'Gap: {total - 68.919154:.6f}')

Baseline total: 70.676102
Target: 68.919154
Gap: 1.756948


In [2]:
# Analyze which N values have the most potential for improvement
# The score formula is side^2/N, so larger N values contribute less per-unit improvement

print('Top 20 N values by score contribution:')
sorted_scores = sorted([(n, scores[n]) for n in range(1, 201)], key=lambda x: -x[1])
for n, score in sorted_scores[:20]:
    side = get_side(configs[n])
    print(f'  N={n:3d}: score={score:.6f}, side={side:.6f}')

Top 20 N values by score contribution:
  N=  1: score=0.661250, side=0.813173
  N=  2: score=0.450779, side=0.949504
  N=  3: score=0.434745, side=1.142031
  N=  5: score=0.416850, side=1.443692
  N=  4: score=0.416545, side=1.290806
  N=  7: score=0.399897, side=1.673104
  N=  6: score=0.399610, side=1.548438
  N=  9: score=0.387415, side=1.867280
  N=  8: score=0.385407, side=1.755921
  N= 15: score=0.379203, side=2.384962
  N= 10: score=0.376630, side=1.940696
  N= 21: score=0.376451, side=2.811667
  N= 20: score=0.376057, side=2.742469
  N= 11: score=0.375736, side=2.033002
  N= 22: score=0.375258, side=2.873270
  N= 16: score=0.374128, side=2.446640
  N= 26: score=0.373997, side=3.118320
  N= 12: score=0.372724, side=2.114873
  N= 13: score=0.372323, side=2.200046
  N= 25: score=0.372144, side=3.050182


In [3]:
# Check what the seshurajup kernel's parallel SA optimizer does
# Key features from the code:
# 1. opt_v3(): Multi-restart optimization with SA
# 2. fractional_translation(): Fine-grained position refinement
# 3. ls_v3(): Local search refinement
# 4. Population-based approach with 3 best solutions maintained

print('Key techniques from seshurajup kernel:')
print('1. sa_v3(): Simulated annealing with temperature schedule')
print('2. ls_v3(): Local search with 300 iterations')
print('3. fractional_translation(): Fine position refinement with 150 iterations')
print('4. opt_v3(): Multi-restart with population of 3 best solutions')
print('5. OpenMP parallelization across N values')
print()
print('The kernel runs with -n 50000 -r 8 (50k iterations, 8 rounds per N)')
print('This is much more intensive than our bbox3 run with -n 10000 -r 16')

Key techniques from seshurajup kernel:
1. sa_v3(): Simulated annealing with temperature schedule
2. ls_v3(): Local search with 300 iterations
3. fractional_translation(): Fine position refinement with 150 iterations
4. opt_v3(): Multi-restart with population of 3 best solutions
5. OpenMP parallelization across N values

The kernel runs with -n 50000 -r 8 (50k iterations, 8 rounds per N)
This is much more intensive than our bbox3 run with -n 10000 -r 16


In [4]:
# Check if the parallel SA optimizer source is available
import os

sa_cpp_path = '/home/code/research/kernels/seshurajup_71-78-jit-parallel-sa-c-tpu-96-cores/71-78-jit-parallel-sa-c-tpu-96-cores.ipynb'
print(f'Seshurajup kernel exists: {os.path.exists(sa_cpp_path)}')

# The C++ code is embedded in the notebook as a %%writefile cell
# We need to extract it and compile it
print('\nNext step: Extract sa_v1_parallel.cpp from the notebook and compile it')

Seshurajup kernel exists: True

Next step: Extract sa_v1_parallel.cpp from the notebook and compile it


In [5]:
# Key insight: The baseline is at a STRONG local optimum
# All our experiments confirm this:
# - bbox3 with 160k iterations: no improvement
# - Deletion cascade: no improvement
# - Random initialization: much worse
# - Lattice without SA: much worse

# The question is: Can we escape this local optimum?
# Options:
# 1. Run much longer optimization (seshurajup's parallel SA)
# 2. Try fundamentally different starting points (egortrushin's lattice + SA)
# 3. Accept that 70.67 may be near-optimal for this problem

print('Analysis of the gap:')
print(f'  Current: 70.676102')
print(f'  Target:  68.919154')
print(f'  Gap:     1.756948 (2.55%)')
print()
print('To close this gap, we need to improve the average per-N score by:')
print(f'  1.756948 / 200 = 0.00878 per N value')
print()
print('This is a significant improvement that requires either:')
print('  1. Finding better configurations for many N values')
print('  2. Finding much better configurations for a few key N values')

Analysis of the gap:
  Current: 70.676102
  Target:  68.919154
  Gap:     1.756948 (2.55%)

To close this gap, we need to improve the average per-N score by:
  1.756948 / 200 = 0.00878 per N value

This is a significant improvement that requires either:
  1. Finding better configurations for many N values
  2. Finding much better configurations for a few key N values
